In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

from IPython.display import Image

In [2]:
tf.__version__

'2.7.0'

# 14.6 텐서플로 추정기
* tf.estimator API
    * 훈련, 예측(추론), 평가와 같은 머신러닝 작업의 기본단계를 캡슐화
    * 확장성 높음
    * 주요 코도를 변경하지 않고 다양한 플랫폼에서 모델을 실행할 수 있도록 지원
    * 널리 사용하는 일부 머신 러닝과 딥러닝 구조를 미리 구현한 추정기 제공


### 14.6.1 특성 열 사용
머신러닝과 딥러닝 애플리케이션에서 다양한 종류의 특성을 볼 수 있다. 연속 특성, 순서가 없는 범주형 특성, 순서가 있는 범주형 특성이 있다. 수치형 데이터는 연속적이거나 이산적일 수 있지만 텐서플로 API에서는 '수치형' 데이터는 특별히 부동 소수 형태의 연속적인 데이터를 나타낸다.

가끔 특성 집합은 다양한 특성 종류가 혼합되어 구성된다. 텐서플로 추정기는 이런 모든 종류의 특성을 다루도록 설계되어 있지만 각 특정이 추정기에서 어떻게 해석되어야 하는지 지정해 주어야 한다.

In [3]:
Image(url='https://git.io/JL56E', width=700)

위 그림에 있는 특성(model year, cylinders, displacement, horsepower, weight, accelecration ,origin)은 Auto MPG 데이터셋 특성이다. 이 데이터셋은 머신러닝 벤치마크 데이터셋은 갤런당 마일(MPG) 단위로 자동차의 연료 효율성을 예측하는 문제이다.

Auto MPG 데이터셋 중 다섯 개의 특성(cylinders, displacement, horsepower, weight, acceleration)은 수치형 (연속적인) 특성이다. model year는 순서가 있는 범주형 특성으로 볼 수 있다. 마지막으로 origin은 순서가 없는 특성이며 각각 US, 유럽, 일본을 의미하는 1, 2, 3 세 개의 값이 가능하다.

In [4]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", 
                                       ("http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"))
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower',
                'Weight', 'Acceleration', 'ModelYear', 'Origin']

df = pd.read_csv(dataset_path, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)
df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [5]:
# 결측값
df.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
ModelYear       0
Origin          0
dtype: int64

In [6]:
# 결측 열 삭제
df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [7]:
import sklearn
from sklearn.model_selection import train_test_split

# 훈련/테스트 분할
df_train, df_test = train_test_split(df, train_size=0.8)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.316933,7.882405,9.0,17.0,22.4,29.0,46.6
Cylinders,313.0,5.501597,1.728577,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,196.017572,106.500183,68.0,98.0,151.0,302.0,455.0
Horsepower,313.0,105.150160,39.467852,46.0,75.0,95.0,130.0,230.0
Weight,313.0,2982.584665,868.540671,1613.0,2220.0,2795.0,3632.0,5140.0
Acceleration,313.0,15.504153,2.785899,8.0,13.7,15.5,17.0,24.8
ModelYear,313.0,75.833866,3.648569,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.584665,0.816509,1.0,1.0,1.0,2.0,3.0


In [8]:
# 수치형 특성
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()

for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std  = train_stats.loc[col_name, 'std']
    df_train_norm.loc[:, col_name] = (df_train_norm.loc[:, col_name] - mean)/std
    df_test_norm.loc[:, col_name] = (df_test_norm.loc[:, col_name] - mean)/std
    
df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,ModelYear,Origin
176,23.0,-0.868690,-0.713779,-0.434535,-0.029457,0.536935,75,2
39,14.0,1.445352,1.455232,1.212375,1.348717,-0.719392,71,1
265,27.5,-0.868690,-0.582324,-0.257175,-0.486546,-0.468127,78,3
93,13.0,1.445352,2.290911,2.783274,2.017655,-1.616768,73,1
149,31.0,-0.868690,-1.098755,-0.966614,-1.131305,0.177984,74,2


numeric_column_names 리스트에 있는 열은 모두 float 타입의 열이다. 이 열들은 연속적인 특성으로 구성될 것이다. 텐서플로의 feature_column 함수를 사용하여 이 연속적인 특성을 텐서플로의 추정기가 다룰 수 있는 특성 열 데이터 구조로 변환한다.

In [9]:
numeric_features = []

for col_name in numeric_column_names:
    numeric_features.append(tf.feature_column.numeric_column(key=col_name))
    
numeric_features

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

훈련 모델의 학습 작업을 단순화하기 위해 model year 정보를 그룹으로 묶어 버킷으로 나눈다.

* 버킷
    * 0: year < 73
    * 1: 73 <= year <= 76
    * 2: 76 <= year <= 79
    * 3: year >= 79

자동차를 세 개의 버킷으로 그룹짓기 위해 먼저 원본 model year를 기본으로 수치 특성을 정의한다. 그다음 이 수치 특성을 [73, 76, 79] 세 개의 분할 기준을 정한 bucketized_column 함수에 전달한다. 이 값은 분할의 오른쪽 기준이다. 이 기준은 한쪽만 닫힌 구간으로 사용된다.

=> (-∞, 73), [73, 76), [76, 79), [79, ∞)

In [10]:
feature_year = tf.feature_column.numeric_column(key="ModelYear")
bucketized_features = []
bucketized_features.append(
    tf.feature_column.bucketized_column(
        source_column=feature_year,
        boundaries=[73, 76, 79]))
print(bucketized_features)

[BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79))]


순서가 없는 범주 특성 Origin을 위해 리스트를 정의한다. 텐서플로에는 범주형 특성 열을 만드는 방법이 여러가지 있다.

* `tf.feature_column.categorical_column_with_vocabulary_list`
 * 가능한 고유 범주 이름을 입력으로 제공할 수 있다.

* `tf.feature_column.categorical_column_with_vocabulary_file`
 * 범주 리스트가 너무 클 경우 사용
 * 모든 범주/단어가 들어 있는 파일을 제공하여 메모리에 모든 가능한 단어 목록을 유지할 필요가 없다.

* `tf.feature_column.categorical_column_with_identity`
 * 특성이 [0, num_categories) 범위의 범주 인덱스에 이미 매핑되어 있을 경우 사용

특성 Origin이 (0, 1, 2가 아니라) 정수 1, 2, 3으로 주어지기 때문에 `categorical_column_with_identity`인 범주 인덱스에 맞지 않는다. 인덱스는 0부터 시작해야하기 때문이다.

In [11]:
feature_origin = tf.feature_column.categorical_column_with_vocabulary_list(
    key='Origin',
    vocabulary_list=[1, 2, 3])

categorical_indicator_features = []
categorical_indicator_features.append(tf.feature_column.indicator_column(feature_origin))
print(categorical_indicator_features)

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]


DNNClassifier와 DNNRegressor 같은 추정기는 소위 밀집 열(dense column)만 받는다. 따라서 다음 단계는 이 범주 특성 열을 밀집 열을 바꾸는 것이다.

* embedding_column 함수
    * 임베딩 열로 바꿈
    * 임베딩 열은 각 인덱스를 float 타입의 랜덤한 수치 벡터로 매핑한다. 이 벡터는 훈련될 수 있다.
* indicator_column 함수
    * 인디케이터 열로 바꿈
    * 인디케이터 열: 범주 인덱스를 원-핫 인코딩된 벡터로 바꾼다.
        * ex, 인덱스 0은 [1, 0, 0]으로 인코딩되고, 인덱스 1은 [0, 1, 0]으로 인코딩되는 식이다.

범주 개수가 많을 때는 범주 개수보다 적은 차원의 임베딩 열을 사용하여 성능을 높일 수 있다.

In [12]:
# 인디케이터 열을 사용하여 범주형 특성 -> 밀집 열
categorical_indicator_features = []
categorical_indicator_features.append(
    tf.feature_column.indicator_column(feature_origin))

### 14.6.2 사전에 준비된 추정기로 머신 러닝 수행
* 단계
 1. 데이터 로딩을 위해 입력 함수 정의
 2. 추정기와 데이터 사이를 연결하기 위해 특성 열 정의
 3. 추정기 객체를 만들거나 케라스 모델을 추정기로 바꾸기
 4. 추정기 사용: train(), evaluate(), predict()

첫 번째 단계에서 데이터를 처리하는 함수를 정의해야한다. 이 함수는 입력 특성과 레이블(MPG)을 담은 튜플로 구성된 텐서플로 데이터셋을 반환한다. 특성은 딕셔너리로 구성되어야 하고 이 딕셔너리 키는 특성 열의 이름과 맞아야한다.

In [13]:
# 첫 번째 단계를 시작하기 위해 훈련 데이터를 위한 입력 함수 정의
def train_input_fn(df_train, batch_size=8):
    df = df_train.copy()
    train_x, train_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(train_x), train_y))
    # 셔플, 반복, 배치
    return dataset.shuffle(1000).repeat().batch(batch_size)

# 확인
ds = train_input_fn(df_train_norm)
batch = next(iter(ds))
print('Keys:', batch[0].keys())
print('ModelYear:', batch[0]['ModelYear'])

Keys: dict_keys(['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'ModelYear', 'Origin'])
ModelYear: tf.Tensor([74 71 73 78 70 74 76 78], shape=(8,), dtype=int64)


두 번째 단계는 특성 열을 정의한다. 연속적인 특성을 담은 리스트, 버킷으로 나눈 특성 열을 위한 리스트, 범주 특성 열을 위한 리스트들을 연결하여 모든 특성 열을 담은 리스트를 만든다.

In [14]:
all_feature_columns = (numeric_features + 
                       bucketized_features + 
                       categorical_indicator_features)
print(all_feature_columns)

[NumericColumn(key='Cylinders', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Displacement', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Horsepower', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Weight', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Acceleration', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), BucketizedColumn(source_column=NumericColumn(key='ModelYear', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(73, 76, 79)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Origin', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]


세 번째 단계에서 추정기를 만든다. MPG 값을 예측하는 것은 전형적인 회귀 문제이므로 tf.estimator.DNNRegressor를 사용한다. 회귀 추정기를 만들 때 특성 열의 리슽와 은닉층의 은닉 유닛 개수를 hidden_units 매개변수에 지정해야 한다.

In [15]:
# 두 개의 은닉층을 사용
# 첫 번째 은닉층은 32개의 유닛, 두 번째 은닉층은 열 개의 유닛을 가진다.
regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    model_dir='models/autompg-dnnregressor/') # model_dir: 모델 파라미터가 저장된 디렉터리를 지정

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


추정기의 장점 중 하나는 훈련하는 동안 자동으로 모델의 체크포인트 파일을 저장하는 것이다. 따라서 예기치 않은 이유로 모델 훈련이 중지되었을 때 마지막으로 저장된 체크포인트를 로드하여 훈련을 이어 갈 수 있다. 체크포인트 파일은 model_dir 매개변수로 지정된 디렉터리에 저장된다. model_dir 매개변수를 지정하지 않았다면 추정기가 랜덤한 임시폴더를 만들어 체크포인트를 저장한다.

세 개의 기본 단계를 끝내야 추정기를 사용하여 훈련, 평가, 예측을 수행할 수있다. 이 회귀 추정기는 train() 메서드로 훈련한다. 이 메서드는 앞에서 정의한 입력 함수가 필요하다.

In [16]:
EPOCHS = 1000
BATCH_SIZE = 8
total_steps = EPOCHS * int(np.ceil(len(df_train) / BATCH_SIZE))
print('훈련 스텝:', total_steps)

regressor.train(
    input_fn=lambda:train_input_fn(df_train_norm, batch_size=BATCH_SIZE),
    steps=total_steps)

훈련 스텝: 40000
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-40000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 40000...
INFO:tensorflow:Saving checkpoints for 40000 into models/autompg-dnnregressor/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 40000...
INFO:tensorflow:loss = 15.040594, step = 40000
INFO:tensorflow:global_step/se

.train()을 호출하면 자동으로 모델 훈련하는 과정동안 체크포인트를 저장한다. 이후 마지막 체크포인트를 다시 불러올 수 있다.

In [17]:
reloaded_regressor = tf.estimator.DNNRegressor(
    feature_columns=all_feature_columns,
    hidden_units=[32, 10],
    warm_start_from='models/autompg-dnnregressor/',
    model_dir='models/autompg-dnnregressor/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/autompg-dnnregressor/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


그 다음 훈련된 모델의 성능을 위해 evaluate() 메서드를 사용할 수 있다.

In [18]:
# 모델을 훈련하고 나서 평가할 때 사용할 테스트 데이터셋을 위한 입력 함수 정의
def eval_input_fn(df_test, batch_size=8):
    df = df_test.copy()
    test_x, test_y = df, df.pop('MPG')
    dataset = tf.data.Dataset.from_tensor_slices((dict(test_x), test_y))
    return dataset.batch(batch_size)

In [19]:
eval_results = reloaded_regressor.evaluate(
    input_fn=lambda:eval_input_fn(df_test_norm, batch_size=8))

for key in eval_results:
    print('{:15s} {}'.format(key, eval_results[key]))
    
print('평균 손실 {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-09T17:33:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-80000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.88494s
INFO:tensorflow:Finished evaluation at 2022-02-09-17:33:11
INFO:tensorflow:Saving dict for global step 80000: average_loss = 11.735458, global_step = 80000, label/mean = 23.956963, loss = 11.722942, prediction/mean = 23.347618
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 80000: models/autompg-dnnregressor/model.ckpt-80000
average_loss    11.735458374023438
label/mean      23.95696258544922
loss            11.722942352294922
prediction/mean 23.347618103027344
global_step     80000
평균 손실 11.7355


마지막으로 새로운 데이터 포인트의 타깃값을 예측하기 위해 predict() 메서드를 사용한다.

실전에서 예측할 때는 입력 함수가 특성만 들어 있는 데이터셋을 반환한다. 새 데이터의 레이블은 구할 수 없다고 가정하기 때문이다.

In [20]:
# 동일 입력 함수를 사용하여 각 샘플 예측
pred_res = regressor.predict(
    input_fn=lambda: eval_input_fn(
        df_test_norm, batch_size=8))
print(next(iter(pred_res)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/autompg-dnnregressor/model.ckpt-80000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([17.716055], dtype=float32)}


* 부스티드 트리 회귀 모델
    * `tf.estimator.BoostedTreesRegressor`

In [21]:
boosted_tree = tf.estimator.BoostedTreesRegressor(
    feature_columns=all_feature_columns,
    n_batches_per_layer=20,
    n_trees=200) # 200개의 트리

boosted_tree.train(
    input_fn=lambda: train_input_fn(df_train_norm, batch_size=BATCH_SIZE))

eval_results = boosted_tree.evaluate(
    input_fn=lambda: eval_input_fn(df_test_norm, batch_size=8))

print(eval_results)
print('평균 손실 {:.4f}'.format(eval_results['average_loss']))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp19faiosg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Plea

### 14.6.3 추정기를 사용하여 MNIST 손글씨 숫자 분류
* 분류 문제를 위한 DNNClassifier 추정기 사용
 * 다층 퍼셉트론을 매우 간단한게 구현 가능

첫 번째 tensorflow_datasets(tdfs) 모듈을 임포트하여 MNIST 데이터셋을 로드하고 모델의 하이퍼파라미터를 지정한다.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
# 설정 단계
# 데이터셋 로드와 하이퍼파라미터
BUFFER_SIZE = 10000 # 데이터를 섞기 위한 BUFFER_SIZE
BATCH_SIZE = 64 # 미니 배치 크기
NUM_EPOCHS = 20 # 훈련 에포크 횟수
steps_per_epoch = np.ceil(60000 / BATCH_SIZE) # 하나의 에포크에서 반복할 횟수 - 무한히 반복하는 데이터셋을 사용하기 때문에 필요

In [3]:
# 입력 이미지와 레이블을 전처리하는 헬퍼 함수 정의
# 입력 이미지는 원래 ([0, 255] 범위인) 'uint8' 타입이기 때문에
# tf.image.convert_image_dtype()을 사용하여 tf.float32 타입으로 변환 => 결국 [0, 1] 범위가 된다
def preprocess(item):
    image = item['image']
    label = item['label']
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.reshape(image, (-1,))
    return {'image-pixels':image}, label[..., tf.newaxis]

In [4]:
# 1단계: 입력 함수 두 개를 정의(하나는 훈련용, 다른 하나는 평가용)
# 훈련용 입력 함수
def train_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_train = datasets['train']

    dataset = mnist_train.map(preprocess)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset.repeat()

# 평가용 입력 함수
def eval_input_fn():
    datasets = tfds.load(name='mnist')
    mnist_test = datasets['test']
    dataset = mnist_test.map(preprocess).batch(BATCH_SIZE)
    return dataset

In [5]:
# 2단계: 특성 열 정의
# 특성 딕셔너리는 'image-pixels' 키 하나만 가지고 있다.
image_feature_column = tf.feature_column.numeric_column(
    key='image-pixels', shape=(28*28)) # 크기가 784(즉,  28*28)인 특성 열 정의 => MNIST 이미지를 일렬로 펼친 것

In [6]:
# 3단계: 새로운 추정기를 만든다.
# 두 개의 은닉층 지정 - 첫 번째 은닉층은 32개의 유닛을 가지고, 두 번째 은닉층은 16개의 유닛을 가진다.
# 또한, n_classes 매개변수로 클래스 개수 지정 - MNIST는 0~9의 숫자 열 개로 구성되어 있으므로 10.
dnn_classifier = tf.estimator.DNNClassifier(
    feature_columns=[image_feature_column],
    hidden_units=[32, 16],
    n_classes=10,
    model_dir='models/mnist-dnn/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/mnist-dnn/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# 4단계: 추정기를 사용하여 훈련, 평가, 예측 수행
dnn_classifier.train(
    input_fn=train_input_fn,
    steps=NUM_EPOCHS * steps_per_epoch)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 18760 into models/mnist-dnn/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 18760...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 18760...


INFO:tensorflow:Loss for final step: 0.36332902.


INFO:tensorflow:Loss for final step: 0.36332902.


In [8]:
eval_result = dnn_classifier.evaluate(
    input_fn=eval_input_fn)

print(eval_result)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2022-02-09T17:39:34


...

INFO:tensorflow:Saving dict for global step 18760: accuracy = 0.8941, average_loss = 0.39317906, global_step = 18760, loss = 0.3932863


INFO:tensorflow:Saving dict for global step 18760: accuracy = 0.8941, average_loss = 0.39317906, global_step = 18760, loss = 0.3932863


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18760: models/mnist-dnn/model.ckpt-18760


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 18760: models/mnist-dnn/model.ckpt-18760


{'accuracy': 0.8941, 'average_loss': 0.39317906, 'loss': 0.3932863, 'global_step': 18760}


### 14.6.4 케라스 모델에서 추정기 만들기
* 개발한 모델을 배포하거나 조직내 다른 구성원에게 모델 공유 가능
* 추정기로 변환하면 분산 훈련이나 체크포인트 자동 저장과 같은 기능 활용 가능
* 모델을 사용하기 쉽다.
* 특성 열과 입력 함수를 지정하여 입력 특성을 해석하기 때문에 혼동을 피할 수 있다.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# XOR 문제
tf.random.set_seed(1)
np.random.seed(1)

# 데이터 생성
x = np.random.uniform(low=-1, high=1, size=(200, 2))
y = np.ones(len(x))
y[x[:, 0] * x[:, 1]<0] = 0

x_train = x[:100, :]
y_train = y[:100]
x_valid = x[100:, :]
y_valid = y[100:]

In [3]:
# 단계 1: 입력 함수 정의
def train_input_fn(x_train, y_train, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices(
        ({'input-features':x_train}, y_train.reshape(-1, )))
    # 셔플, 반복, 배치
    return dataset.shuffle(100).repeat().batch(batch_size)

def eval_input_fn(x_test, y_test=None, batch_size=8):
    if y_test is None:
        dataset = tf.data.Dataset.from_tensor_slices(
            {'input-features':x_test})
    else:
        dataset = tf.data.Dataset.from_tensor_slices(
            ({'input-features':x_test}, y_test.reshape(-1, 1)))
    return dataset.batch(batch_size)

In [4]:
# 단계 2: 특성 열 정의
features = [tf.feature_column.numeric_column(
    key='input-features:', shape=(2,))]
features

[NumericColumn(key='input-features:', shape=(2,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [5]:
# 단계 3: Sequential 클래스를 사용하여 모델 정의
# 모델 입력에 이믈을 부여하기 위해 tf.keras.layers.Input으로 입력층 추가
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,), name='input-features'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(units=4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 12        
                                                                 
 dense_1 (Dense)             (None, 4)                 20        
                                                                 
 dense_2 (Dense)             (None, 4)                 20        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

my_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model,
    model_dir='models/estimator-for-XOR/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': 'models/estimator-for-XOR/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# 단계 4: 추정기를 사용하여 모델 훈련 및 검증 데이터셋으로 평가, 예측
num_epochs = 200
batch_size = 2
steps_per_epoch = np.ceil(len(x_train) / batch_size)

my_estimator.train(
    input_fn=lambda: train_input_fn(x_train, y_train, batch_size),
    steps=num_epochs * steps_per_epoch)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='models/estimator-for-XOR/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: models/estimator-for-XOR/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 8 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into models/estimator-for-XOR/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.75971186, step = 0
INFO:tensorflow:global_step/sec: 432.892
INFO:tensorflow:loss = 0.6824059, step = 100 (0.239 sec)

In [8]:
my_estimator.evaluate(
    input_fn=lambda: eval_input_fn(x_valid, y_valid, batch_size))

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.7/dist-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Done calling model_fn.


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


INFO:tensorflow:Starting evaluation at 2022-02-09T17:48:05
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/estimator-for-XOR/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.63080s
INFO:tensorflow:Finished evaluation at 2022-02-09-17:48:06
INFO:tensorflow:Saving dict for global step 10000: binary_accuracy = 0.93, global_step = 10000, loss = 0.09085727
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: models/estimator-for-XOR/model.ckpt-10000


{'binary_accuracy': 0.93, 'global_step': 10000, 'loss': 0.09085727}